In [1]:
import pandas as pd
import numpy as np


In [2]:
train_df = pd.read_csv("../Data/train_clean.csv")
test_df  = pd.read_csv("../Data/test_clean.csv")

print(train_df.shape)
print(test_df.shape)


(614, 13)
(367, 12)


In [3]:
cat_cols = [
    "Gender",
    "Married",
    "Dependents",
    "Education",
    "Self_Employed",
    "Property_Area"
]

num_cols = [
    "ApplicantIncome",
    "CoapplicantIncome",
    "LoanAmount",
    "Loan_Amount_Term",
    "Credit_History"
]


In [4]:
for col in cat_cols:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)
    test_df[col].fillna(test_df[col].mode()[0], inplace=True)

for col in num_cols:
    train_df[col].fillna(train_df[col].median(), inplace=True)
    test_df[col].fillna(test_df[col].median(), inplace=True)


C:\Users\G15_GAMING\AppData\Local\Temp\ipykernel_11748\2731979517.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_df[col].mode()[0], inplace=True)
C:\Users\G15_GAMING\AppData\Local\Temp\ipykernel_11748\2731979517.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

In [5]:
# Total income
train_df["Total_Income"] = train_df["ApplicantIncome"] + train_df["CoapplicantIncome"]
test_df["Total_Income"]  = test_df["ApplicantIncome"] + test_df["CoapplicantIncome"]

# Debt to income ratio
train_df["Debt_Income_Ratio"] = train_df["LoanAmount"] / train_df["Total_Income"]
test_df["Debt_Income_Ratio"]  = test_df["LoanAmount"] / test_df["Total_Income"]


In [6]:
full_df = pd.concat([train_df, test_df], axis=0)

full_df = pd.get_dummies(
    full_df,
    columns=cat_cols,
    drop_first=True
)

train_df = full_df.iloc[:len(train_df)]
test_df  = full_df.iloc[len(train_df):]


In [7]:
X = train_df.drop(["Risk", "Loan_ID"], axis=1)
y = train_df["Risk"].map({
    "Low Risk": 0,
    "High Risk": 1
})


In [8]:
from sklearn.preprocessing import StandardScaler
import pickle

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [9]:
with open("../model/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


In [10]:
X_processed = pd.DataFrame(X_scaled, columns=X.columns)

X_processed.to_csv("../Data/X_processed.csv", index=False)
y.to_csv("../Data/y.csv", index=False)

test_df.to_csv("../Data/test_processed.csv", index=False)
